# LangChain: Agents

## Outline:

* Using built in LangChain tools: Math and Wikipedia
* Defining your own tools

# Setup

In [ ]:
!pip -q install openai
!pip -q install langchain
!pip -q install langchain_openai # >= 0.1.0

# !pip -q install docarray

In [ ]:
!pip install langchain_experimental
!pip install langchainhub

In [3]:
!python -c "import openai; print('openai:', openai.__version__)"
!python -c "import langchain; print('langchain:', langchain.__version__)"

openai: 1.7.1
langchain: 0.1.0


In [4]:
#API KEY 저장을 위한 os 라이브러리 호출
import os
from google.colab import userdata

#OPENAI API키 저장
#API KEY 발급 페이지: https://platform.openai.com/docs/guides/gpt/completions-api
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [193]:
import langchain

from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType, AgentExecutor, create_react_agent, create_structured_chat_agent

# old version
# from langchain.agents.agent_toolkits import create_python_agent
# from langchain.tools.python.tool import PythonREPLTool
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL

from langchain_openai import ChatOpenAI, OpenAI # >= 0.1.0
# from langchain.chat_models import ChatOpenAI # for < 0.1.0

# Built-in LangChain Tools

- `llm-math`, `wikipedia`

In [217]:
llm_model = "gpt-3.5-turbo-1106"

llm = ChatOpenAI(temperature=0, model=llm_model)

## Setup Tools

In [98]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=ab5bc1141be7302a6f589914252a9ce8a51c8976a54c4a355a47b371eb81c9e4
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [205]:
tools = load_tools(["llm-math","wikipedia"], llm=llm)

In [206]:
agent = initialize_agent(
    tools=tools,
    llm=llm,
    # REACT 프레임워크를 따르는 제로샷으로 동작하는 에이전트
    # 즉 질문에 대한 [질의,응답] 예제없이 Thought, Action, Obervation, ... 을 반복하는 에이전트
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    # 에이전트가 루프를 돌다가 에러가 나면 에러 메지지를 다시 llm에게 넣고
    # 적절히 에러를 수정하게 하는 옵션
    handle_parsing_errors=True,
    verbose = True)

#### [참고] handle_parsing_errors 옵션 확인

In [207]:
from langchain import hub

In [208]:
prompt = hub.pull("hwchase17/react")
prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

In [209]:
prompt.input_variables
# https://www.pinecone.io/learn/series/langchain/langchain-agents/
# agent_scratchpad 에 대한 설명있음
# 과거 Thought와 Action의 결과를 모두 지정하는 변수

['agent_scratchpad', 'input', 'tool_names', 'tools']

In [210]:
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [213]:
gpt_inst = OpenAI(temperature=0)

# REACT 에이전트 생성
agent = create_react_agent(gpt_inst, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, tools=tools,
    # True로 하면 Action이 없으면 Action이 없다는 메세지를 다시 LLM에 다시 넣고
    # Thought를 기반으로 Action을 찾아냄
    handle_parsing_errors=True,
    verbose=True
)

agent_executor.invoke(
    #  프롬프트에 Action: Wikipedia가 있으면 첫번째 체인에서 Action이 출력되지 않는 문제가 있음
    {"input": "What is Leo DiCaprio's middle name?\n\nAction: Wikipedia"}
)

# ValueError: An output parsing error occurred.
# In order to pass this error back to the agent and have it try again,
# pass `handle_parsing_errors=True` to the AgentExecutor.
# This is the error: Could not parse LLM output: ` I should search for "Leo DiCaprio" on Wikipedia
# Action Input: Leo DiCaprio`



> Entering new AgentExecutor chain...
 I should use Wikipedia to search for information about Leo DiCaprio
Action Input: Leo DiCaprioInvalid Format: Missing 'Action:' after 'Thought:I should use Wikipedia to search for information about Leo DiCaprio
Action: Wikipedia
Action Input: Leo DiCaprioPage: Leonardo DiCaprio
Summary: Leonardo Wilhelm DiCaprio (; Italian: [diˈkaːprjo]; born November 11, 1974) is an American actor and film producer. Known for his work in biographical and period films, he is the recipient of numerous accolades, including an Academy Award, a British Academy Film Award, and three Golden Globe Awards. As of 2019, his films have grossed over $7.2 billion worldwide, and he has been placed eight times in annual rankings of the world's highest-paid actors.
Born in Los Angeles, DiCaprio began his career in the late 1980s by appearing in television commercials. In the early 1990s, he had recurring roles in various television shows, such as the sitcom Parenthood, and had 

{'input': "What is Leo DiCaprio's middle name?\n\nAction: Wikipedia",
 'output': 'George'}

## llm-math

In [214]:
# deprecated
# agent= initialize_agent(
#     tools=tools,
#     llm=llm,
#     agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
#     handle_parsing_errors=True,
#     verbose = True)

agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, tools=tools,
    handle_parsing_errors=True,
    verbose=True
)

In [215]:
agent_executor.invoke({"input":"What is the 25% of 300?"})



> Entering new AgentExecutor chain...
I need to calculate 25% of 300.
Action: Calculator
Action Input: 300 * 0.25Answer: 75.0The final answer is 75.0
Final Answer: 75.0

> Finished chain.


{'input': 'What is the 25% of 300?', 'output': '75.0'}

In [218]:
langchain.debug=True
agent_executor.invoke({"input":"What is the 25% of 300?"})
langchain.debug=False

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "What is the 25% of 300?"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence] Entering Chain run with input:
{
  "input": "What is the 25% of 300?",
  "intermediate_steps": []
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign<agent_scratchpad>] Entering Chain run with input:
{
  "input": "What is the 25% of 300?",
  "intermediate_steps": []
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign<agent_scratchpad> > 4:chain:RunnableParallel<agent_scratchpad>] Entering Chain run with input:
{
  "input": "What is the 25% of 300?",
  "intermediate_steps": []
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign<agent_scratchpad> > 4:chain:RunnableParallel<agent_scratchpad> > 5:chain:RunnableLambda] Entering Chain run with input:
{
  "input": "What is the 25% of 300?",
  "intermediate

## wikipedia

In [219]:
question = "Tom M. Mitchell is an American computer scientist \
and the Founders University Professor at Carnegie Mellon University (CMU)\
what book did he write?"

result = agent_executor.invoke({'input':question})



> Entering new AgentExecutor chain...
I should use Wikipedia to find out what book Tom M. Mitchell wrote.
Action: Wikipedia
Action Input: Tom M. Mitchell

/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Page: Tom M. Mitchell
Summary: Tom Michael Mitchell (born August 9, 1951) is an American computer scientist and the Founders University Professor at Carnegie Mellon University (CMU). He is a founder and former Chair of the Machine Learning Department at CMU. Mitchell is known for his contributions to the advancement of machine learning, artificial intelligence, and cognitive neuroscience and is the author of the textbook Machine Learning. He is a member of the United States National Academy of Engineering since 2010. He is also a Fellow of the American Academy of Arts and Sciences, the American Association for the Advancement of Science and a Fellow and past President of the Association for the Advancement of Artificial Intelligence. In October 2018, Mitchell was appointed as the Interim Dean of the School of Computer Science at Carnegie Mellon.

Page: Ensemble learning
Summary: In statistics and machine learning, ensemble methods use multiple learning algorithms to obtain better predi

# Python Agent

In [229]:
python_agent = create_python_agent(
    llm,
    tool=PythonREPLTool(),
    # 파이썬 에이전트에서도 이거 없으면 가끔 출력 파싱 에러남
    handle_parsing_errors=True,
    verbose=True
)

In [230]:
customer_list = [
    # first name, last name
    ["Harrison", "Chase"],
    ["Lang", "Chain"],
    ["Dolly", "Too"],
    ["Elle", "Elem"],
    ["Geoff","Fusion"],
    ["Trance","Former"],
    ["Jen","Ayai"]
]

In [231]:
result = python_agent.invoke({'input':f"""Sort these customers by last name and then first name and print the output: {customer_list}"""})



> Entering new AgentExecutor chain...
I can use the sorted() function in Python to sort the list of customers by last name and then first name.
Action: Python_REPL
Action Input: customers = [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]
sorted_customers = sorted(customers, key=lambda x: (x[1], x[0]))
print(sorted_customers)
Observation: [['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]

Thought:The customers are now sorted by last name and then first name.
Final Answer: [['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]

> Finished chain.


In [232]:
result

{'input': "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]",
 'output': "[['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]"}

In [233]:
eval(result['output'])

[['Jen', 'Ayai'],
 ['Lang', 'Chain'],
 ['Harrison', 'Chase'],
 ['Elle', 'Elem'],
 ['Trance', 'Former'],
 ['Geoff', 'Fusion'],
 ['Dolly', 'Too']]

## 활용 예제


- ref: https://realpython.com/python-practice-problems/

- A Caesar cipher is a simple substitution cipher in which each letter of the plain text is substituted with a letter found by moving n places down the alphabet. For example, assume the input plain text is the following:
`abcd xyz`

- If the shift value, n, is 4, then the encrypted text would be the following:
`efgh bcd`

- You are to write a function that accepts two arguments, a plain-text message and a number of letters to shift in the cipher. The function will return an encrypted string with all letters transformed and all punctuation and whitespace remaining unchanged.


In [234]:
input_text = 'dfq %^$^& mrpe'

shift_num = 3

def caesar_cipher(input_text: str, shift_num: int) -> str:
    #                0                        26
    alphabet = list('abcdefghijklmnopqrstuvwxyz')
    index = []
    for c in input_text:
        try:
            index.append(alphabet.index(c))
        except ValueError:
            index.append(c)

    shifted = []
    for  i in index:
        if type(i) == int:
            shifted.append(alphabet[(i+shift_num) % 26])
        else:
            shifted.append(i)

    return ''.join(shifted)

caesar_cipher(input_text, shift_num)

'git %^$^& push'

In [235]:
# langchain.debug=True

#프롬프트를 작성하고 에이전트에게 작업을 시켜보기
result = python_agent.invoke({'input':f"""
A Caesar cipher is a simple substitution cipher in which each letter of
the plain text is substituted with a letter found by moving n places down the alphabet.

For example, assume the input plain text is the following:

abcd xyz

If the shift value, n, is 4, then the encrypted text would be the following:

efgh bcd


Shift the string given in 'Input String' by the number given by 'Shift Value'.
The 'Input String' and 'Shift Value' are separated by ###.

Input String: ###{input_text}###
Shift Value: ###{shift_num}###

The input string is entered in lowercase letters of the alphabet and other characters.
Non-alphabetical characters are output without processing.
"""
})

# langchain.debug=False



> Entering new AgentExecutor chain...
I need to write a Python function to implement the Caesar cipher with the given shift value.

Action: Python_REPL
Action Input:
```python
def caesar_cipher(input_string, shift):
    result = ""
    for char in input_string:
        if char.isalpha():
            shift_amount = shift % 26
            if char.islower():
                shifted_char = chr(((ord(char) - 97 + shift_amount) % 26) + 97)
            else:
                shifted_char = chr(((ord(char) - 65 + shift_amount) % 26) + 65)
            result += shifted_char
        else:
            result += char
    return result

caesar_cipher("dfq %^$^& mrpe", 3)
```
Observation: 
Thought:I now know the final answer
Final Answer: gis %^$^& pusr

> Finished chain.


In [236]:
result['output']

'gis %^$^& pusr'

# Define your own tool

In [237]:
from langchain.agents import tool
from datetime import date

In [238]:
@tool
def time(text: str) -> str:
    """
    Returns todays date, use this for any questions related to knowing todays date.
    The input should always be an empty string and this function will always return todays date.
    Any date mathmatics should occur outside this function.
    """
    return str(date.today())

In [239]:
tools = tools+[time]

agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, tools=tools,
    # True로 하면 Action이 없으면 Action이 없다는 메세지를 다시 LLM에 다시 넣고
    # Thought를 기반으로 Action을 찾아냄
    handle_parsing_errors=True,
    verbose=True
)

In [240]:
result = agent_executor({"input":"whats the date today?"})




> Entering new AgentExecutor chain...
I should use the time function to get today's date.
Action: time
Action Input: ""2024-01-11I now know the date today.
Final Answer: The date today is January 11, 2024.

> Finished chain.


In [241]:
result

{'input': 'whats the date today?',
 'output': 'The date today is January 11, 2024.'}

### 활용 예제

- 스트링 쉬프트 함수를 툴로 만들기

In [242]:
tools

[Tool(name='Calculator', description='Useful for when you need to answer questions about math.', func=<bound method Chain.run of LLMMathChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['question'], template='Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n${{Output of running the code}}\n```\nAnswer: ${{Answer}}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n```text\n37593 * 67\n```\n...numexpr.evaluate("37593 * 67")...\n```output\n2518731\n```\nAnswer: 2518731\n\nQuestion: 37593^(1/5)\n```text\n37593**(1/5)\n```\n...numexpr.evaluate("37593**(1/5)")...\n```output\n8.222831614237718\n```\nAnswer: 8.222831614237718\n\nQuestion: {question}\n'), llm=ChatOpenAI(client=<openai.resources.chat.completions.Comp

In [243]:

# 함수의 디스크립션을 직접 적어보기

@tool
def caesar_cipher(input_text: str, shift_num: int) -> str:
    """
    Shift the given string by n places.

    For example, assume the input plain text is the following:
    abcd xyz

    If the shift value, n, is 4, then the encrypted text would be the following:
    efgh bcd
    """

    alphabet = list('abcdefghijklmnopqrstuvwxyz')
    index = []
    for c in input_text:
        try:
            index.append(alphabet.index(c))
        except ValueError:
            index.append(c)

    shifted = []
    for  i in index:
        if type(i) == int:
            shifted.append(alphabet[(i+shift_num) % 26])
        else:
            shifted.append(i)

    return ''.join(shifted)


In [244]:
# 툴에 추가
tools=tools+[caesar_cipher]

tools

[Tool(name='Calculator', description='Useful for when you need to answer questions about math.', func=<bound method Chain.run of LLMMathChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['question'], template='Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n${{Output of running the code}}\n```\nAnswer: ${{Answer}}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n```text\n37593 * 67\n```\n...numexpr.evaluate("37593 * 67")...\n```output\n2518731\n```\nAnswer: 2518731\n\nQuestion: 37593^(1/5)\n```text\n37593**(1/5)\n```\n...numexpr.evaluate("37593**(1/5)")...\n```output\n8.222831614237718\n```\nAnswer: 8.222831614237718\n\nQuestion: {question}\n'), llm=ChatOpenAI(client=<openai.resources.chat.completions.Comp

In [245]:
# 구조화된 체팅 프롬프트
prompt = hub.pull("hwchase17/structured-chat-agent")

prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['tool_names', 'tools'], template='Respond to the human as helpfully and accurately as possible. You have access to the following tools:\n\n{tools}\n\nUse a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).\n\nValid "action" values: "Final Answer" or {tool_names}\n\nProvide only ONE action per $JSON_BLOB, as shown:\n\n```\n{{\n  "action": $TOOL_NAME,\n  "action_input": $INPUT\n}}\n```\n\nFollow this format:\n\nQuestion: input question to answer\nThought: co

In [246]:
# 에이전트 만들기
agent = create_structured_chat_agent(
    llm, tools, prompt
)

# 에이전트 실행기
agent_executor = AgentExecutor(
    agent=agent, tools=tools,
    handle_parsing_errors=True,
    verbose=True
)

In [247]:
# 실행!
result = agent_executor({"input":"Shift the text 'dfq %^$^& mrpe' by 3"})



> Entering new AgentExecutor chain...
Action:
```{
  "action": "caesar_cipher",
  "action_input": {
    "input_text": "dfq %^$^& mrpe",
    "shift_num": 3
  }
}```git %^$^& push```
{
  "action": "Final Answer",
  "action_input": "git %^$^& push"
}
```

> Finished chain.


In [248]:
result['output']

'git %^$^& push'